In [1]:
!pip install sconce -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido

In [2]:
from collections import defaultdict, OrderedDict

import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
import torch.optim as optim

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"

Load the Pre-Trained Model Weights

In [3]:
from google.colab import drive
drive.mount('/content/drive')
model_path = "drive/MyDrive/Efficientml/Efficientml.ai/vgg.cifar.pretrained.pth"

Mounted at /content/drive


In [4]:

class VGG(nn.Module):
  ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, 10)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
    x = self.backbone(x)

    # avgpool: [N, 512, 2, 2] => [N, 512]
    x = x.mean([2, 3])

    # classifier: [N, 512] => [N, 10]
    x = self.classifier(x)
    return x


#load the pretrained model

model = VGG().cuda()
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

Setup the Dataset

In [5]:
image_size = 32
transforms = {
    "train": transforms.Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:

    dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
    )

dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )




100%|██████████| 170498071/170498071 [00:03<00:00, 48771323.02it/s]


Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


sconce Configurations

In [6]:
from sconce import sconce


sconces = sconce()
sconces.model= model
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-gmp"
sconces.prune_mode = "GMP" # Supports Automated Pruning Ratio Detection


Train and Validated the Model on the given dataset

In [7]:
# Train the model
sconces.train()
# Evaludate the model
sconces.evaluate()

Epoch:1 Train Loss: 0.00000 Validation Accuracy: 92.92585


92.92585170340682

Magic Happens here: Compress the model(GMP pruning is set as the prune mode[sconces.prune_mode] above)

In [8]:
# Compress the model granuarly
sconces.compress()


Original Dense Model Size Model=35.20 MiB


Original Model Validation Accuracy: 92.92585170340682 %
Granular-Magnitude Pruning


Sparsity for each Layer: {'backbone.conv0.weight': 0.45000000000000007, 'backbone.conv1.weight': 0.7000000000000002, 'backbone.conv2.weight': 0.5000000000000001, 'backbone.conv3.weight': 0.25000000000000006, 'backbone.conv4.weight': 0.3500000000000001, 'backbone.conv5.weight': 0.6000000000000002, 'backbone.conv6.weight': 0.25000000000000006, 'backbone.conv7.weight': 0.30000000000000004, 'classifier.weight': 0.8500000000000002}



Pruned Model has size=21.94 MiB(non-zeros) = 62.31% of Original model size


Epoch:1 Train Loss: 0.00000 Validation Accuracy: 92.67535


Epoch:2 Train Loss: 0.00000 Validation Accuracy: 92.74549


Epoch:3 Train Loss: 0.00000 Validation Accuracy: 92.81563


Epoch:4 Train Loss: 0.00000 Validation Accuracy: 92.78557


Epoch:5 Train Loss: 0.00000 Validation Accuracy: 92.82565



 ................. Comparison Table  .................
                Original        Pruned          Reduction Ratio
Latency (ms)    34800.0         33400.0         1.0            
MACs (M)        606             606             1.0            
Param (M)       9.23            5.75            1.6            
Accuracies (%)  92.926          92.826          -0.1           
Fine-Tuned Sparse model has size=21.94 MiB = 62.31% of Original model size
Fine-Tuned Pruned Model Validation Accuracy: 92.82565130260521


**Channel-Wise Pruning**

In [ ]:
sconces = sconce()
sconces.model= model
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-cwp"
sconces.prune_mode = "CWP" # Supports Automated Pruning Ratio Detection


In [9]:
# Compress the model Channel-Wise
sconces.compress()


Original Dense Model Size Model=21.94 MiB


Original Model Validation Accuracy: 92.82565130260521 %



 Channel-Wise Pruning
Sparsity for each Layer: {'backbone.conv0.weight': 0.1, 'backbone.conv1.weight': 0.1, 'backbone.conv2.weight': 0.15000000000000002, 'backbone.conv3.weight': 0.15000000000000002, 'backbone.conv4.weight': 0.25000000000000006, 'backbone.conv5.weight': 0.20000000000000004, 'backbone.conv6.weight': 0.40000000000000013}



Pruned Model has size=12.81 MiB(non-zeros) = 58.41% of Original model size


Epoch:1 Train Loss: 0.00000 Validation Accuracy: 89.83968


Epoch:2 Train Loss: 0.00000 Validation Accuracy: 89.97996


Epoch:3 Train Loss: 0.00000 Validation Accuracy: 90.21042


Epoch:4 Train Loss: 0.00000 Validation Accuracy: 90.39078


Epoch:5 Train Loss: 0.00000 Validation Accuracy: 90.59118



 ................. Comparison Table  .................
                Original        Pruned          Reduction Ratio
Latency (ms)    38700.0         21200.0         1.8            
MACs (M)        606             410             1.5            
Param (M)       5.75            5.44            1.1            
Accuracies (%)  92.826          90.591          -2.234         
Fine-Tuned Sparse model has size=20.76 MiB = 94.62% of Original model size
Fine-Tuned Pruned Model Validation Accuracy: 90.59118236472946


Notice that intially,


*   **Dense Model** has a size of *35.20MiB* and accuracy of *92.89%*.
*   **Post Pruning(GMP) Pruned Model** size *23.18MiB* with accuracy of *65.85%*.
*   Upon **fine-tuning the Prune Model**, we have the final pruned model size of *23.18MiB* with an accuracy of *93.19%*.


| Metric                        | Original | Pruned  | Reduction Ratio |
|-------------------------------|----------|---------|-----------------|
| Latency (ms)                  | 19900.0  | 19900.0 | 1.0             |
| MACs (M)                      | 606      | 606     | 1.0             |
| Param (M)                     | 9.23     | 9.23    | 1.0             |
| Fine-Tuned Sparse Model Size  | -        | 23.18 MiB | 65.85%         |
| Fine-Tuned Pruned Model Validation Accuracy | -    | 93.196% | -   |






Spiking Neural Network Compression


In [10]:
!pip install snntorch -q

In [11]:
# Import snntorch libraries
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt
from torch import optim

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools



<ipython-input-11-b898cb6c07c2>:4: DeprecationWarning: The module snntorch.backprop will be deprecated in  a future release. Writing out your own training loop will lead to substantially faster performance.
  from snntorch import backprop


In [12]:

# Event Drive Data

# dataloader arguments
batch_size = 128
data_path = "./data/mnist"

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define a transform
transform = transforms.Compose(
    [
        transforms.Resize((28, 28)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,)),
    ]
)

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, drop_last=True
)
test_loader = DataLoader(
    mnist_test, batch_size=batch_size, shuffle=True, drop_last=True
)

# Set you Dataloader
dataloader = {}
dataloader["train"] = train_loader
dataloader["test"] = test_loader
sconces.dataloader = dataloader


100%|██████████| 9912422/9912422 [00:00<00:00, 84768984.25it/s]


Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 18909724.29it/s]


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 42180357.38it/s]


Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4731875.00it/s]


Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



In [13]:
sconces = sconce()

sconces.snn = True



spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
# Load your snn Model
snn_model = nn.Sequential(
    nn.Conv2d(1, 12, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
    nn.Conv2d(12, 64, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
    nn.Flatten(),
    nn.Linear(64 * 4 * 4, 10),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True),
).to(device)
snn_pretrained_model_path = "drive/MyDrive/Efficientml/Efficientml.ai/snn_model.pth"
snn_model.load_state_dict(torch.load(snn_pretrained_model_path))  # Model Definition
sconces.model = snn_model

sconces.optimizer = optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)

sconces.criterion = SF.ce_rate_loss()

sconces.epochs = 10  # Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "snn-gmp"  # Define your experiment name here
sconces.prune_mode = "GMP"
sconces.num_finetune_epochs = 1



In [14]:
sconces.compress()

AttributeError: ignored